In [ ]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
from config import conf as cfg
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import plotly.graph_objects as go
from tools import btools
from IPython.display import HTML
from pandarallel import pandarallel # 导入pandaralle

pandarallel.initialize(progress_bar=False)
%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
col_names=['query', 'target', 'qtmscore', 'ttmscore', 'alntmscore', 'fident', 'lddt', 'prob', 'alnlen', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits']
df1 = pd.read_csv('/hpcfs/fhome/shizhenkun/codebase/RXNRECer/temp/aa.m8',  sep='\t', index_col=None, names=col_names)
df1 

,query,target,qtmscore,ttmscore,alntmscore,fident,lddt,prob,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits
0,AF-J8D1N4-F1-model_v4,AF-A3Q9F0-F1-model_v4.pdb,0.2453,0.67460,0.67460,0.024,0.4332,0.045,165,95,13,16,165,1,114,0.4360,23
1,AF-J8D1N4-F1-model_v4,AF-Q2GI11-F1-model_v4.pdb,0.2529,0.65650,0.65650,0.048,0.4152,0.051,165,94,17,13,159,3,122,0.4325,24
2,AF-J8D1N4-F1-model_v4,AF-Q5FF38-F1-model_v4.pdb,0.2602,0.64830,0.64830,0.065,0.4093,0.057,167,96,17,11,159,1,125,0.4320,25
3,AF-J8D1N4-F1-model_v4,AF-Q5HCB3-F1-model_v4.pdb,0.2534,0.65510,0.65510,0.052,0.4223,0.051,170,94,15,12,162,1,122,0.4316,24
4,AF-J8D1N4-F1-model_v4,AF-B7L097-F1-model_v4.pdb,0.2620,0.64410,0.64410,0.052,0.4218,0.057,171,99,19,10,160,1,128,0.4297,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,AF-J8GG30-F1-model_v4,AF-Q99MS7-F1-model_v4.pdb,0.2232,0.08022,0.08022,0.007,0.6080,0.034,1072,146,10,63,359,784,1712,0.1485,21
2711,AF-J8GG30-F1-model_v4,AF-Q10172-F1-model_v4.pdb,0.2238,0.08187,0.08187,0.004,0.4760,0.034,1438,167,16,14,471,252,1404,0.1475,21
2712,AF-J8GG30-F1-model_v4,ESM-Q06VD6,0.2116,0.08922,0.08922,0.007,0.5748,0.029,1255,144,15,120,366,320,1480,0.1463,20
2713,AF-J8GG30-F1-model_v4,AF-A2AL36-F1-model_v4.pdb,0.2246,0.06501,0.06501,0.006,0.2581,0.034,2305,165,34,22,539,282,2247,0.1398,21


In [ ]:
col_names=['query',  'qtmscore', 'ttmscore', 'alntmscore','target']
df = pd.read_csv('/hpcfs/fhome/shizhenkun/codebase/RXNRECer/temp/aa.m8',  sep='\t', index_col=None, names=col_names)
df 

,query,qtmscore,ttmscore,alntmscore,target
0,AF-J8G8J6-F1-model_v4,0.5640,0.7328,0.7328,AF-A1RW47-F1-model_v4.pdb
1,AF-J8G8J6-F1-model_v4,0.5617,0.7219,0.7219,AF-B3SAE4-F1-model_v4.pdb
2,AF-J8G8J6-F1-model_v4,0.5765,0.6759,0.6759,AF-Q8PW97-F1-model_v4.pdb
3,AF-J8G8J6-F1-model_v4,0.5598,0.5523,0.5523,AF-Q49YA5-F1-model_v4.pdb
4,AF-J8G8J6-F1-model_v4,0.5428,0.5791,0.5791,AF-Q9FAW5-F1-model_v4.pdb
...,...,...,...,...,...
2710,AF-J8D1N4-F1-model_v4,0.2249,0.2537,0.2537,AF-Q645Y8-F1-model_v4.pdb
2711,AF-J8D1N4-F1-model_v4,0.2410,0.2211,0.2211,AF-Q499V3-F1-model_v4.pdb
2712,AF-J8D1N4-F1-model_v4,0.2786,0.1723,0.1723,AF-P44246-F1-model_v4.pdb
2713,AF-J8D1N4-F1-model_v4,0.2935,0.1255,0.1255,AF-Q8C052-F1-model_v4.pdb


In [8]:
df.sort_values(by=['query','alntmscore'], ascending=False)

,query,qtmscore,ttmscore,alntmscore,target
1400,AF-J8HQ06-F1-model_v4,0.3978,0.7808,0.7808,ESM-P15403
1428,AF-J8HQ06-F1-model_v4,0.3778,0.7350,0.7350,ESM-P22169
1541,AF-J8HQ06-F1-model_v4,0.2930,0.7013,0.7013,AF-Q7NVK0-F1-model_v4.pdb
1542,AF-J8HQ06-F1-model_v4,0.2982,0.6952,0.6952,AF-Q48TL1-F1-model_v4.pdb
1547,AF-J8HQ06-F1-model_v4,0.2979,0.6940,0.6940,AF-Q9CHX5-F1-model_v4.pdb
...,...,...,...,...,...
2711,AF-J8D1N4-F1-model_v4,0.2410,0.2211,0.2211,AF-Q499V3-F1-model_v4.pdb
2709,AF-J8D1N4-F1-model_v4,0.2789,0.2147,0.2147,AF-B1HTV8-F1-model_v4.pdb
2712,AF-J8D1N4-F1-model_v4,0.2786,0.1723,0.1723,AF-P44246-F1-model_v4.pdb
2713,AF-J8D1N4-F1-model_v4,0.2935,0.1255,0.1255,AF-Q8C052-F1-model_v4.pdb


In [1]:
import pandas as pd

In [2]:
rxn = pd.read_feather('http://172.16.8.95:9000/bucket-brem-web/web_reactions.feather')

In [7]:
cpd = pd.read_feather('http://172.16.8.95:9000/bucket-brem-web/web_compounds.feather')
cpd.head(3)

,cpd_id,cpd_name,iupac_name,molecular_formula,inchi,inchikey,smiles,synonyms,mol,ref_chebi,ref_pubchem,ref_unichem,num_of_reactions,funcion_group,funcion_group_smarts,canonical_smiles,check
0,CPD:1,3'-sialyllactose,-,C23H39NO19,InChI=1S/C23H39NO19/c1-7(29)24-13-8(30)2-23(22...,-,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](O...,3'-sialyllactose|#35890-38-1|#sialyllactose|#3...,-,-,CID:123914,-,0,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H1;!R].[C;D2;H...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H1&!R].[C&D2&H...,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](O...,False
1,CPD:2,1-butanethiol,-,C4H10S,"InChI=1S/C4H10S/c1-2-3-4-5/h5H,2-4H2,1H3",-,CCCCS,1-butanethiol|#butanethiol|#butane-1-thiol|#10...,-,-,CID:8012,-,1,[C;D1;H3;!R].[C;D2;H2;!R].[SX2;H1;!R],[C&D1&H3&!R].[C&D2&H2&!R].[S&X2&H1&!R],CCCCS,False
2,CPD:3,N-(5-aminopentyl)acetamide,-,C7H16N2O,"InChI=1S/C7H16N2O/c1-7(10)9-6-4-2-3-5-8/h2-6,8...",-,CC(=O)NCCCCCN,n-(5-aminopentyl)acetamide|#32343-73-0|#monoac...,-,-,CID:189087,-,3,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H0;!$(CO)](=[O...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H0&!$(CO)]=[O&...,CC(=O)NCCCCCN,False


In [9]:
cpd.head(1000)

,cpd_id,cpd_name,iupac_name,molecular_formula,inchi,inchikey,smiles,synonyms,mol,ref_chebi,ref_pubchem,ref_unichem,num_of_reactions,funcion_group,funcion_group_smarts,canonical_smiles,check
0,CPD:1,3'-sialyllactose,-,C23H39NO19,InChI=1S/C23H39NO19/c1-7(29)24-13-8(30)2-23(22...,-,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](O...,3'-sialyllactose|#35890-38-1|#sialyllactose|#3...,-,-,CID:123914,-,0,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H1;!R].[C;D2;H...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H1&!R].[C&D2&H...,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](O...,False
1,CPD:2,1-butanethiol,-,C4H10S,"InChI=1S/C4H10S/c1-2-3-4-5/h5H,2-4H2,1H3",-,CCCCS,1-butanethiol|#butanethiol|#butane-1-thiol|#10...,-,-,CID:8012,-,1,[C;D1;H3;!R].[C;D2;H2;!R].[SX2;H1;!R],[C&D1&H3&!R].[C&D2&H2&!R].[S&X2&H1&!R],CCCCS,False
2,CPD:3,N-(5-aminopentyl)acetamide,-,C7H16N2O,"InChI=1S/C7H16N2O/c1-7(10)9-6-4-2-3-5-8/h2-6,8...",-,CC(=O)NCCCCCN,n-(5-aminopentyl)acetamide|#32343-73-0|#monoac...,-,-,CID:189087,-,3,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H0;!$(CO)](=[O...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H0&!$(CO)]=[O&...,CC(=O)NCCCCCN,False
3,CPD:4,D-alt,-,C6H12O6,InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2...,-,OC[C@H]1OC(O)[C@@H](O)[C@H](O)[C@@H]1O,d-alt|#d-altropyranose|#d-altropyranoside|#d-a...,-,-,CID:441032,-,2,[C;D2;H2;!R].[C;D3;H1;R].[C][O;D2;H0;!$(OC=O);...,[C&D2&H2&!R].[C&D3&H1&R].C[O&D2&H0&!$(OC=O)&!$...,OC[C@H]1OC(O)[C@@H](O)[C@H](O)[C@@H]1O,False
4,CPD:5,A very-long-chain 3-oxoacyl-coa,-,C24H33N7O18P3SR,,-,*C(=O)CC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)CO...,a very-long-chain 3-oxoacyl-coa,-,CHEBI:90725,-,-,0,[C;D1;H3;!R].[C;D2;H2;!R].[C;D3;H1;!R].[C;D4;H...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D3&H1&!R].[C&D4&H...,*C(=O)CC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)CO...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,CPD:996,"4-(hydroxylamino)-n,n-dimethylaniline",-,C8H12N2O,InChI=1S/C8H12N2O/c1-10(2)8-5-3-7(9-11)4-6-8/h...,-,CN(C)c1ccc(NO)cc1,"4-(hydroxylamino)-n,n-dimethylaniline|#4-(n,n-...",-,-,CID:46173248,-,0,[C;D1;H3;!R].c1ccccc1.[N;D3;H0;!R;!$(NC=O)],[C&D1&H3&!R].c1ccccc1.[N&D3&H0&!R&!$(NC=O)],CN(C)c1ccc(NO)cc1,False
996,CPD:997,1-methylpiperidin-4-ol,-,C6H13NO,"InChI=1S/C6H13NO/c1-7-4-2-6(8)3-5-7/h6,8H,2-5H...",-,CN1CCC(O)CC1,1-methylpiperidin-4-ol|#106-52-5|#n-methyl-4-p...,-,-,CID:66048,-,2,[C;D1;H3;!R].[C;D2;H2;R].[C;D3;H1;R].[O;D1;H1;...,[C&D1&H3&!R].[C&D2&H2&R].[C&D3&H1&R].[O&D1&H1&...,CN1CCC(O)CC1,False
997,CPD:998,Strictosidine aglycone,-,C21H24N2O4,InChI=1S/C21H24N2O4/c1-3-12-15(16(20(24)26-2)1...,-,C=C[C@H]1[C@H](O)OC=C(C(=O)OC)[C@H]1C[C@@H]1NC...,strictosidine aglycone|#85925-13-9|#methyl (2r...,-,-,CID:439972,-,1,[C;D1;H3;!R].[C;D2;H2;!R].[C;D2;H2;R].[C;D3;H1...,[C&D1&H3&!R].[C&D2&H2&!R].[C&D2&H2&R].[C&D3&H1...,C=C[C@H]1[C@H](O)OC=C(C(=O)OC)[C@H]1C[C@@H]1NC...,False
998,CPD:999,N-terminal l-valine residue,-,C5H11NO,,-,*C(=O)[C@@H]([NH3+])C(C)C,n-terminal l-valine residue|#(2s)-2-azaniumyl-...,-,CHEBI:64741,-,-,0,[C;D1;H3;!R].[C;D3;H1;!R].[C;D3;H0;!$(CO)](=[O...,[C&D1&H3&!R].[C&D3&H1&!R].[C&D3&H0&!$(CO)]=[O&...,*C(=O)[C@@H]([NH3+])C(C)C,False


In [4]:
rxn.columns

Index(['reaction_id', 'equation_string', 'equation_string_new', 'equation',
       'equation_smiles', 'reversible', 'natural', 'ec', 'organism',
       'protein_refs', 'ref_enymemap', 'num_of_enzymes',
       'rxn_atom_mapped_enzymemap', 'rxn_center', 'lb_rxn_center_s1',
       'rxn_atom_mapped_enzymemap_single', 'rxn_atom_mapped_rxnmapper',
       'lb_rxn_center_s1_enzymemap', 'rxn_pic',
       'rxn_atom_mapped_enzymemap_rdt', 'num_of_reactions'],
      dtype='object')

In [10]:
relation = pd.read_feather('http://172.16.8.95:9000/bucket-brem-web/relation/web_reaction_enzyme_relation.feather')

In [12]:
rxn

,reaction_id,equation_string,equation_string_new,equation,equation_smiles,reversible,natural,ec,organism,protein_refs,...,num_of_enzymes,rxn_atom_mapped_enzymemap,rxn_center,lb_rxn_center_s1,rxn_atom_mapped_enzymemap_single,rxn_atom_mapped_rxnmapper,lb_rxn_center_s1_enzymemap,rxn_pic,rxn_atom_mapped_enzymemap_rdt,num_of_reactions
0,RXN:1,valeraldehyde + NADH = 1-pentanol + NAD+,valeraldehyde + NADH + h+ = 1-pentanol + NAD+,CPD:9339 + CPD:1059 + CPD:53737>>CPD:3673 + CP...,CCCCC=O.NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP...,{r},False,"1.1.1.21,1.1.1.1,1.1.1.202","Rhodococcus ruber , Bos taurus, Euonymus japon...","A3MVR8 , Q59477",...,2,[CH3:1][CH2:2][CH2:3][CH2:4][CH:5]=[O:6].[H+]....,[C:5].[O:6].[C:10].[C:11].[N:12].[C:48].[C:49]...,COCCNCCC>>COCCNCCC,[CH3:1][CH2:2][CH2:3][CH2:4][CH:5]=[O:6].[H+]....,[CH3:1][CH2:2][CH2:3][CH2:4][CH:5]=[O:6].[NH2:...,[C:6].[O:8] + [C:1].[C:2].[N:3].[C:4].[C:5].[C...,[['pic/reaction/0f62ab64d2071a23759836fc022252...,[O:1]=[C:2]([OH:3])[C:4](=[O:5])[CH2:6][CH2:7]...,NaN
1,RXN:2,pyridinecarboxaldehyde + NAD(P)H = pyridylmeth...,NADH + 2-pyridinecarboxaldehyde + h+ = NAD+ + ...,CPD:1059 + CPD:4054 + CPD:53737>>CPD:53738 + C...,NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)O...,{undefined},False,1.1.1.21,"Homo sapiens, Euonymus japonicus",,...,0,[H+].[NH2:1][C:2](=[O:3])[C:4]1=[CH:5][N:6]([C...,[C:4].[C:5].[N:6].[C:42].[C:43].[C:44].[O:45]....,CCNCCCOC>>CCNCCCOC,[H+].[NH2:1][C:2](=[O:3])[C:4]1=[CH:5][N:6]([C...,[NH2:1][C:2](=[O:3])[C:4]1=[CH:44][N:8]([C@@H:...,[C:1].[C:7].[N:8].[C:2].[C:3].[C:4] + [O:5].[C...,[['pic/reaction/366beba58e60902bd4e2070e4880ae...,[O:1]=[C:2]([OH:3])[C@@H:4]([NH2:5])[CH2:6][CH...,NaN
2,RXN:3,S-adenosyl-L-methionine + L-Lys = S-adenosyl-L...,S-adenosyl-L-methionine + l-lysine = N(2)-meth...,CPD:19654 + CPD:2643>>CPD:48378 + CPD:2891 + C...,C[S+](CC[C@H](N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(...,{undefined},False,"2.1.1.301,2.1.1.127","Pisum sativum , Streptomyces sp.",Q43088,...,1,[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C...,[C:1].[S:2].[N:34]>>[C:1].[N:34].[S:2],CSN>>CNS,[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C...,[CH3:1][S+:26]([CH2:25][C@H:24]1[O:23][C@@H:22...,[C:2].[S:3] + [N:4]>>[C:2].[N:4] + [H:1] + [S:3],[['pic/reaction/63fb9ff0ebae9695411c0f3c8f177f...,[O:1]=[C:2]([OH:3])[C@@H:4]([NH2:5])[CH2:6][CH...,NaN
3,RXN:4,S-adenosyl-L-methionine + methyl-L-Lys = S-ade...,N(2)-methyl-L-lysine + S-adenosyl-L-methionine...,CPD:48378 + CPD:19654>>CPD:57088 + CPD:2891 + ...,CN[C@@H](CCCCN)C(=O)O.C[S+](CC[C@H](N)C(=O)O)C...,{undefined},False,2.1.1.127,Pisum sativum,Q43088,...,1,[CH3:12][S+:13]([CH2:14][CH2:15][C@H:16]([NH2:...,[C:12].[S:13].[N:8]>>[N:8].[C:12].[S:13],CSN>>NCS,[CH3:12][S+:13]([CH2:14][CH2:15][C@H:16]([NH2:...,[NH2:2][CH2:3][CH2:4][CH2:5][CH2:6][C@H:7]([NH...,[C:2].[S:3] + [N:4]>>[N:4].[C:2] + [H:1] + [S:3],[['pic/reaction/629a18db4bd177b087e8d3d6624c7e...,[O:1]=[C:2]([OH:3])[C@@H:4]([NH2:5])[CH2:6][CH...,NaN
4,RXN:5,S-adenosyl-L-methionine + (S)-norlaudanosoline...,S-adenosyl-L-methionine + (S)-norlaudanosoline...,CPD:19654 + CPD:20620>>CPD:21770 + CPD:2891 + ...,C[S+](CC[C@H](N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(...,{undefined},True,2.1.1.128,"Coptis japonica, Papaver somniferum, Eschschol...",,...,0,[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C...,[C:1].[S:2].[O:41]>>[C:1].[O:41].[S:2],CSO>>COS,[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C...,[CH3:1][S+:37]([CH2:36][C@H:35]1[O:34][C@@H:33...,[C:2].[S:3] + [O:4]>>[C:2].[O:4] + [H:1] + [S:3],[['pic/reaction/63fb9ff0ebae9695411c0f3c8f177f...,[O:1]=[C:2]([OH:3])[C@@H:4]([NH2:5])[CH2:6][CH...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30217,RXN:54632,S-adenosyl-L-methionine + 10-hydroxydihydrosan...,10-hydroxydihydrosanguinarine + S-adenosyl-L-m...,CPD:24921 + CPD:19654>>CPD:7427 + CPD:2891 + C...,CN1Cc2c3c(cc(O)c2-c2ccc4cc5c(cc4c21)OCO5)OCO3....,{undefined},True,2.1.1.119,Eschscholzia californica,,...,0,[CH3:1][N:2]1[CH2:3][c:4]2[c:5]3[c:6]([cH:7][c...,[O:9].[C:27].[

In [16]:
proteins =pd.read_feather('http://172.16.8.95:9000/bucket-brem-web/protein/web_proteins.feather')
proteins

,uniprot_id,reviewed,ec,name_entry,name_protein,name_gene,organism,seq_length,pubmed_id,date_create,date_modified,date_sequence_modified,seq,rhea_id,kcat,active_site,binding_site,ph,isenzyme,num_of_reactions
0,A0A1B0GTW7,reviewed,3.4.24.-,CIROP_HUMAN,Ciliated left-right organizer metallopeptidase...,CIROP LMLN2,Homo sapiens (Human),788.0,12508121; 15489334; 34903892,2018-09-12,2023-09-13,2016-10-05,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...,,,"ACT_SITE 306; /evidence=""ECO:0000255|PROSITE-P...","BINDING 305; /ligand=""Zn(2+)""; /ligand_id=""ChE...",,True,
1,A1L3X0,reviewed,2.3.1.199,ELOV7_HUMAN,Elongation of very long chain fatty acids prot...,ELOVL7,Homo sapiens (Human),281.0,15489334; 14702039; 17974005; 19826053; 209379...,2007-12-04,2023-11-08,2007-02-06,MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVY...,RHEA:32727 RHEA:35327 RHEA:36475 RHEA:35379 RH...,BIOPHYSICOCHEMICAL PROPERTIES: Kinetic parame...,"ACT_SITE 150; /note=""Nucleophile""; /evidence=""...","BINDING 124; /ligand=""3-oxoeicosanoyl-CoA""; /l...",,True,
2,A2RUC4,reviewed,1.14.11.42,TYW5_HUMAN,tRNA wybutosine-synthesizing protein 5 (hTYW5)...,TYW5 C2orf60,Homo sapiens (Human),315.0,14702039; 15815621; 15489334; 20739293; 20972222,2007-11-13,2023-11-08,2007-03-06,MAGQHLPVPRLEGVSREQFMQHLYPQRKPLVLEGIDLGPCTSKWTV...,RHEA:37899,,,"BINDING 106; /ligand=""2-oxoglutarate""; /ligand...",,True,
3,A4GXA9,reviewed,3.1.22.-,EME2_HUMAN,Probable crossover junction endonuclease EME2 ...,EME2,Homo sapiens (Human),379.0,17289582; 12693554; 11157797,2008-02-05,2023-11-08,2014-05-14,MARVGPGRAGVSCQGRGRGRGGSGQRRPPTWEISDSDAEDSAGSEA...,,,,,,True,
4,A5PLL7,reviewed,1.14.19.77,PDES1_HUMAN,Plasmanylethanolamine desaturase 1 (EC 1.14.19...,PEDS1 KUA PDES TMEM189,Homo sapiens (Human),270.0,11076860; 11280764; 14702039; 11780052; 154893...,2008-02-26,2023-11-08,2010-11-02,MAGAENWPGQQLELDEDEASCCRWGAQHAGARELAALYSPGKRLQE...,RHEA:22956 RHEA:61960 RHEA:61964 RHEA:61968,,,,,True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282513,Q8GTQ6,unreviewed,2.5.1.16,Q8GTQ6_MALDO,spermidine synthase (EC 2.5.1.16),MdSPDS1,Malus domestica (Apple) (Pyrus malus),335.0,12655406,2003-03-01,2024-01-24,2011-09-21,MADESVAGSADFPVKRPREEEENGSAAAASAVSMEPDGGKEPDSVS...,RHEA:12721,-,"ACT_SITE 206; /note=""Proton acceptor""; /eviden...",-,-,True,-
282514,A6XP71,unreviewed,-,A6XP71_MORAP,Malic enzyme,mce2,Mortierella alpina (Oleaginous fungus) (Mortie...,669.0,17600047,2007-08-21,2024-01-24,2007-08-21,MALSSLSTLPRSSAARSFFLQKGNNSCLHSSFSCCYSTKISIPRHT...,-,-,"ACT_SITE 184; /note=""Proton donor""; /evidence=...","BINDING 249; /ligand=""substrate""; /evidence=""E...",-,True,-
282515,Q8U3V1,unreviewed,-,Q8U3V1_PYRFU,PIG-L family deacetylase,PF0354,Pyrococcus furiosus (strain ATCC 43587 / DSM 3...,267.0,10430560; 24702737; 26057790,2002-06-01,2024-01-24,2002-06-01,MFENVSTFEEAFNKLLKEVLEFNLENPFEDAKKVICIEPHPDDCAI...,-,-,-,"BINDING 40; /ligand=""Zn(2+)""; /ligand_id=""ChEB...",-,True,-
282516,Q6F7Q1,unreviewed,3.1.2.-,Q6F7Q1_ACIAD,Acyl-CoA thioesterase II (EC 3.1.2.-),tesB ACIAD3231,Acinetobacter baylyi (strain ATCC 33305 / BD41...,290.0,15514110,2004-07-19,2024-01-24,2004-07-19,MNTLTQELVELLSLEKLEENLYRGMSRNLVGKRVFGGQVLGQALRA...,-,-,-,-,-,True,-


In [22]:
rxn = pd.read_feather('/hpcfs/fhome/shizhenkun/codebase/public_data/public_data_wsi23/web/web_reactions.feather')
rxn

,reaction_id,equation_string,equation,equation_smiles,ec,num_of_enzymes
0,RHEA:10008,[protein]-dithiol + a hydroperoxide = [protein...,CHEBI:29950 + CHEBI:35924 = CHEBI:50058 + CHEB...,*OO.*N[C@@H](CS)C(*)=O>>*O.*N[C@@H](CSSC[C@H](...,-,8
1,RHEA:10012,(R)-6-hydroxynicotine + H2O + O2 = 6-hydroxyps...,CHEBI:58413 + CHEBI:15377 + CHEBI:15379 = CHEB...,O=O.O.C[NH+]1CCC[C@@H]1c1ccc(O)nc1>>OO.C[NH2+]...,1.5.3.6,1
2,RHEA:10016,H2O + O-sinapoylcholine = choline + E-sinapate...,CHEBI:15377 + CHEBI:16353 = CHEBI:15354 + CHEB...,O.COc1cc(/C=C/C(=O)OCC[N+](C)(C)C)cc(OC)c1O>>C...,3.1.1.49,1
3,RHEA:10020,H2O + L-saccharopine + NADP(+) = (S)-2-amino-6...,CHEBI:15377 + CHEBI:57951 + CHEBI:58349 = CHEB...,[NH3+][C@@H](CCCC[NH2+][C@@H](CCC(=O)[O-])C(=O...,1.5.1.10,4
4,RHEA:10024,L-lysyl-[histone] + S-adenosyl-L-methionine = ...,CHEBI:29969 + CHEBI:59789 = CHEBI:15378 + CHEB...,*N[C@@H](CCCC[NH3+])C(*)=O.C[S+](CC[C@H]([NH3+...,-,24
...,...,...,...,...,...,...
19959,brnx:9245,glutaraldehyde + NAD(P)+ + H2O = glutarate + N...,cid:3485 + cid:5886 + cid:962 = cid:743 + cid...,O.NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=...,1.2.1.26,1
19960,brnx:9246,2-methoxybenzaldehyde + NADPH + H+ = 2-methoxy...,cid:5884 + cid:8658 = cid:5886 + cid:69154,COc1ccccc1C=O.NC(=O)C1=CN([C@@H]2O[C@H](COP(=O...,1.1.1.71,1
19961,brnx:9247,serotonin + UDP-glucuronate = UDP + serotonin ...,cid:5202 + cid:17473 = cid:6031 + cid:12962...,NCCc1c[nH]c2ccc(O)cc12.O=C(O)[C@H]1O[C@H](OP(=...,2.4.1.17,11
19962,brnx:9248,ATP + N-acetyl-D-muramate = ADP + N-acetyl-alp...,cid:5957 + cid:16738680 = cid:6022 + cid:12...,CC(=O)N[C@H]1C(O)O[C@H](CO)[C@@H](O)[C@@H]1O[C...,2.7.1.221,1


In [19]:
rxn[rxn.ec=='4.1.2.61']

,reaction_id,equation_string,equation,equation_smiles,ec,num_of_enzymes
4389,RHEA:36307,"(E)-caffeoyl-CoA + H2O = 3,4-dihydroxybenzalde...",CHEBI:87136 + CHEBI:15377 = CHEBI:50205 + CHEB...,CC(C)(COP(=O)([O-])OP(=O)([O-])OC[C@H]1O[C@@H]...,4.1.2.61,1
9125,RHEA:62412,(E)-feruloyl-CoA + H2O = acetyl-CoA + vanillin,CHEBI:87305 + CHEBI:15377 = CHEBI:57288 + CHEB...,O.COc1cc(/C=C/C(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(...,4.1.2.61,1
9126,RHEA:62416,(E)-4-coumaroyl-CoA + H2O = 4-hydroxybenzaldeh...,CHEBI:85008 + CHEBI:15377 = CHEBI:17597 + CHEB...,O.CC(C)(COP(=O)([O-])OP(=O)([O-])OC[C@H]1O[C@@...,4.1.2.61,1
14591,brnx:3877,feruloyl-CoA + H2O = 4-hydroxy-3-methoxyphenyl...,cid:44229137 + cid:962 = cid:11966199,O.COc1cc(/C=C/C(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(...,4.1.2.61,1
16261,brnx:5547,3-hydroxy-3-(4-hydroxy-3-methoxyphenyl)propano...,cid:11966199 = cid:1183 + cid:444493,COc1cc(C(O)CC(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)...,4.1.2.61,1
17619,brnx:6905,feruloyl-CoA + H2O = 4-hydroxy-3-methoxybenzal...,cid:44229137 + cid:962 = cid:1183 + cid:444493,O.COc1cc(/C=C/C(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(...,4.1.2.61,4


In [23]:
rxn1 = pd.read_feather('http://172.16.8.95:9000/bucket-sreaction-paper/reaction/web_reactions.feather')
rxn1

,reaction_id,equation_string,equation,equation_smiles,ec,num_of_enzymes
0,RHEA:10008,[protein]-dithiol + a hydroperoxide = [protein...,CHEBI:29950 + CHEBI:35924 = CHEBI:50058 + CHEB...,*OO.*N[C@@H](CS)C(*)=O>>*O.*N[C@@H](CSSC[C@H](...,-,8
1,RHEA:10012,(R)-6-hydroxynicotine + H2O + O2 = 6-hydroxyps...,CHEBI:58413 + CHEBI:15377 + CHEBI:15379 = CHEB...,O=O.O.C[NH+]1CCC[C@@H]1c1ccc(O)nc1>>OO.C[NH2+]...,1.5.3.6,1
2,RHEA:10016,H2O + O-sinapoylcholine = choline + E-sinapate...,CHEBI:15377 + CHEBI:16353 = CHEBI:15354 + CHEB...,O.COc1cc(/C=C/C(=O)OCC[N+](C)(C)C)cc(OC)c1O>>C...,3.1.1.49,1
3,RHEA:10020,H2O + L-saccharopine + NADP(+) = (S)-2-amino-6...,CHEBI:15377 + CHEBI:57951 + CHEBI:58349 = CHEB...,[NH3+][C@@H](CCCC[NH2+][C@@H](CCC(=O)[O-])C(=O...,1.5.1.10,4
4,RHEA:10024,L-lysyl-[histone] + S-adenosyl-L-methionine = ...,CHEBI:29969 + CHEBI:59789 = CHEBI:15378 + CHEB...,*N[C@@H](CCCC[NH3+])C(*)=O.C[S+](CC[C@H]([NH3+...,-,24
...,...,...,...,...,...,...
19959,brnx:9245,glutaraldehyde + NAD(P)+ + H2O = glutarate + N...,cid:3485 + cid:5886 + cid:962 = cid:743 + cid...,O.NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=...,1.2.1.26,1
19960,brnx:9246,2-methoxybenzaldehyde + NADPH + H+ = 2-methoxy...,cid:5884 + cid:8658 = cid:5886 + cid:69154,COc1ccccc1C=O.NC(=O)C1=CN([C@@H]2O[C@H](COP(=O...,1.1.1.71,1
19961,brnx:9247,serotonin + UDP-glucuronate = UDP + serotonin ...,cid:5202 + cid:17473 = cid:6031 + cid:12962...,NCCc1c[nH]c2ccc(O)cc12.O=C(O)[C@H]1O[C@H](OP(=...,2.4.1.17,11
19962,brnx:9248,ATP + N-acetyl-D-muramate = ADP + N-acetyl-alp...,cid:5957 + cid:16738680 = cid:6022 + cid:12...,CC(=O)N[C@H]1C(O)O[C@H](CO)[C@@H](O)[C@@H]1O[C...,2.7.1.221,1


In [25]:
relation1 = pd.read_feather('http://172.16.8.95:9000/bucket-sreaction-paper/reaction/web_reaction_enzyme_relation.feather')
relation1

,reaction_id,uniprot_id,organism
0,RHEA:10008,O17433,Dirofilaria immitis (Canine heartworm)
1,RHEA:10008,O34564,Bacillus subtilis (strain 168)
2,RHEA:10008,P0C0L2,Escherichia coli (strain K12)
3,RHEA:10008,P0C0L3,Shigella flexneri
4,RHEA:10008,P23161,Clostridium pasteurianum
...,...,...,...
329103,brnx:9247,P35503,Homo sapiens (Human)
329104,brnx:9247,P16662,Homo sapiens (Human)
329105,brnx:9247,P35504,Homo sapiens (Human)
329106,brnx:9248,Q88QT3,Pseudomonas putida (strain ATCC 47054 / DSM 61...


In [26]:
protein1 = pd.read_feather('http://172.16.8.95:9000/bucket-sreaction-paper/reaction/web_proteins.feather')
protein1

,uniprot_id,num_of_reactions,name,ec,organism,reviewed,pubmed_id,seq,latest_update
0,Q842E2,85,L-amino acid ligase (Probable enzyme),None,Pseudomonas syringae,unreviewed,23770908,MTQAKENILVVVDGYSSGSQLPTLMAESGWKCVHVSSSANPPEYYL...,2022-08-03
1,P27169,66,Serum paraoxonase/arylesterase 1 (PON 1) (EC 3...,3.1.1.2; 3.1.1.81; 3.1.8.1,Homo sapiens (Human),reviewed,1657140; 7916578; 8393742; 8393745; 8812495; 9...,MAKLIALTLLGMGLALFRNHQSSYQTRLNALREVQPVELPNCNLVK...,2022-10-12
2,A3RE51,66,Diacylglycerol O-acyltransferase (EC 2.3.1.20),2.3.1.20,Marinobacter nauticus (Marinobacter hydrocarbo...,unreviewed,17351040,MKRLGTLDASWLAVESEDTPMHVGTLQIFSLPEGAPETFLRDMVTR...,2022-10-12
3,A0A4P8GNL5,60,Putrescine aminotransferase,None,Pseudomonas putida (Arthrobacter siderocapsula...,unreviewed,nan,MSTNNPQTREWQTLSGEHHLAPFSDYKQLKEKGPRIITKAQGVHLW...,2022-08-03
4,P58137,58,Acyl-coenzyme A thioesterase 8 (Acyl-CoA thioe...,3.1.2.1; 3.1.2.11; 3.1.2.2; 3.1.2.27; 3.1.2.3;...,Mus musculus (Mouse),reviewed,11673457; 15489334; 16141203; 21183079,MSAPEGLGDAHGDADRGDLSGDLRSVLVTSVLNLEPLDEDLYRGRH...,2022-10-12
...,...,...,...,...,...,...,...,...,...
234193,Q4P2Q7,1,"3-hydroxyanthranilate 3,4-dioxygenase (EC 1.13...",1.13.11.6,Ustilago maydis (strain 521 / FGSC 9021) (Corn...,reviewed,17080091,MPFPLPLNFPKWLSENEHLLQPPVGNFCLFRTRDYTVMAVGGPNAR...,2022-08-03
234194,Q4UT95,1,"3-hydroxyanthranilate 3,4-dioxygenase (EC 1.13...",1.13.11.6,Xanthomonas campestris pv. campestris (strain ...,reviewed,15899963,MLVPPINLHAWVEQHRHLLKPPVGNKCIQQDGFIIMIVGGPNARTD...,2022-08-03
234195,Q4WCF1,1,"3-hydroxyanthranilate 3,4-dioxygenase 1 (EC 1....",1.13.11.6,Neosartorya fumigata (strain ATCC MYA-4609 / A...,reviewed,16372009,MLPPALNIPKWLEENSHLLQPPVNNYCVYHPSSPATAGYTVMIVGG...,2022-08-03
234196,Q4WZC3,1,"3-hydroxyanthranilate 3,4-dioxygenase 2 (EC 1....",1.13.11.6,Neosartorya fumigata (strain ATCC MYA-4609 / A...,reviewed,16372009,MATLNPLSWVTWLAENEDKLRPPVNNYCLYQGNDFILMAVGGPNER...,2022-08-03


In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json

# --- 示例：若本地无 cavas，可自行替换为 logging 或其他错误处理机制 ---
try:
    import cavas
except ImportError:
    cavas = None


def display_error_with_cavas(error_message: str):
    """
    用于演示如何通过 cavas 显示错误信息的函数。
    若没有 cavas，请替换为其它可行的报错处理方案 (print/logging等)。
    """
    if cavas:
        cavas.show_error(error_message)
    else:
        print("[Error]", error_message)


def fetch_main_table(url: str):
    """
    请求主页面并解析表格，返回表格中提取的数据列表。
    """
    try:
        resp = requests.get(url)
        resp.raise_for_status()
    except requests.exceptions.RequestException as e:
        display_error_with_cavas(f"请求主页面失败: {e}")
        return []

    soup = BeautifulSoup(resp.text, "html.parser")

    # 查找目标表格 (此处示例中没有指定确切的 ID 或 class，
    # 请根据该站点实际 HTML 结构选择相应的 selector)
    table = soup.find("table")
    if not table:
        display_error_with_cavas("未能在页面中找到 <table>，请检查页面结构。")
        return []

    # 收集表格数据
    results = []

    # 找到所有行，假设第 0 行是表头，数据从第 1 行开始
    rows = table.find_all("tr")
    if len(rows) <= 1:
        display_error_with_cavas("表格行数过少，可能没有数据。")
        return []

    # 第一行为表头
    # 这里如果需要列名可自行解析, 也可直接跳过
    header_cells = [th.get_text(strip=True) for th in rows[0].find_all(["th", "td"])]

    for row in rows[1:]:
        cells = row.find_all("td")
        # 如果列数过少，需要加一些判断
        if not cells:
            continue

        # 下面是示例逻辑，根据实际页面的列结构去解析
        # 假设表中至少有5列（仅举例），可适当调整
        # 也可能需要用到 row.find(...) 精确定位子链接
        # ----------------------------------------------------------------
        # 这里的列名 & 数据列，请根据网页实际内容做修改 !!!
        # ----------------------------------------------------------------
        col1_text = cells[0].get_text(strip=True) if len(cells) >= 1 else ""
        col2_text = cells[1].get_text(strip=True) if len(cells) >= 2 else ""
        col3_text = cells[2].get_text(strip=True) if len(cells) >= 3 else ""
        col4_text = cells[3].get_text(strip=True) if len(cells) >= 4 else ""
        col5_text = cells[4].get_text(strip=True) if len(cells) >= 5 else ""

        # 如果某列中有子链接，我们再获取其 href
        # 例如假设在第 0 列（或其他列）中包含链接:
        sub_link_tag = cells[0].find("a")
        sub_link = sub_link_tag["href"] if sub_link_tag else None
        if sub_link and not sub_link.startswith("http"):
            # 若 href 是相对路径，需要拼接主域名
            sub_link = "https://regprecise.lbl.gov/" + sub_link.lstrip("/")

        row_data = {
            "col1": col1_text,
            "col2": col2_text,
            "col3": col3_text,
            "col4": col4_text,
            "col5": col5_text,
            "sub_link": sub_link
        }
        results.append(row_data)

    return results


def fetch_sub_link_data(sub_link: str):
    """
    请求子链接页面并提取需要的信息（示例）。
    可根据子页面真实的 DOM 结构灵活调整。
    """
    if not sub_link:
        return {}

    try:
        resp = requests.get(sub_link)
        resp.raise_for_status()
    except requests.exceptions.RequestException as e:
        display_error_with_cavas(f"请求子链接失败: {e}")
        return {}

    soup = BeautifulSoup(resp.text, "html.parser")

    # 在子页面中，可能需要解析更多表格、文本、基因名称等
    # 以下仅作示例，具体提取需参照实际页面结构
    detail_div = soup.find("div", {"class": "ContentBlock"})
    detail_text = detail_div.get_text(strip=True) if detail_div else "无详细说明"

    return {
        "sub_detail": detail_text
    }


def main():
    # 目标 URL
    url = "https://regprecise.lbl.gov/propagated_genome.jsp?pgp_collection_id=6&genome_id=98"

    print("开始爬取主页面表格数据...")
    main_table_data = fetch_main_table(url)
    if not main_table_data:
        print("主表格数据为空，程序结束。")
        return

    # 存放最终数据
    final_data = []

    # 在对表格中的每一行进行处理时，显示进度
    for row_item in tqdm(main_table_data, desc="处理子链接", unit="条"):
        sub_link = row_item.get("sub_link")
        if sub_link:
            # 如果存在子链接，则进一步解析
            sub_data = fetch_sub_link_data(sub_link)
            # 将子链接获取到的内容合并到 row_item
            row_item.update(sub_data)

        final_data.append(row_item)

    # 将结果写入 JSON 文件
    output_file = "regprecise_data.json"
    try:
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(final_data, f, ensure_ascii=False, indent=4)
        print(f"爬取完成，结果已保存到: {output_file}")
    except Exception as e:
        display_error_with_cavas(f"写入 JSON 文件失败: {e}")


if __name__ == "__main__":
    main()


开始爬取主页面表格数据...


处理子链接: 100%|██████████| 42/42 [00:00<00:00, 570099.57条/s]

爬取完成，结果已保存到: regprecise_data.json
